# Cross Validation

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = False
c.wandb.group = "LB"
c.wandb.dir = "../../cache/"
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretraind_dir = "../../datasets/trainings"

In [4]:
pretrained = f"""
- dir: {pretraind_dir}/2022-02-25_22-16-54/fold0/
  model: ump_lstm
- dir: {pretraind_dir}/2022-02-25_22-16-56/fold1/
  model: ump_lstm
- dir: {pretraind_dir}/2022-02-25_22-16-58/fold2/
  model: ump_lstm
- dir: {pretraind_dir}/2022-02-25_22-17-00/fold3/
  model: ump_lstm
- dir: {pretraind_dir}/2022-02-25_22-17-02/fold4/
  model: ump_lstm
"""

_pretrained = f"""
"""

c.params.pretrained = OmegaConf.create(pretrained)

In [5]:
log.info(OmegaConf.to_yaml(c))

2022-02-26 20:46:16,903 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ../../cache/
  group: LB
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ${hydra:runtime.cwd}/../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ${settings.dirs.input}preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 440
  n_class: 10
  preprocess:
  - remove_china_shock
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_nam

In [6]:
import os

import pandas as pd
import src.utils as utils
from src.get_score import record_result
from tqdm.notebook import tqdm

In [7]:
run = utils.setup_wandb(c)

In [8]:
train = pd.read_feather("../../inputs/train.f")
train = train.loc[:, ["row_id", "time_id", "target"]]
train.set_index("row_id", inplace=True)

In [9]:
preds_col = []

# 各 OOF の結果を読み取り
for n, training in tqdm(enumerate(c.params.pretrained), total=len(c.params.pretrained)):
    preds_col.append(f"preds{n}")
    oof_df = pd.read_feather(os.path.join(training.dir.rsplit("/", 2)[0], "oof_df.f")).set_index("row_id")

    if training.model == "lightgbm":
        train[f"preds{n}"] = oof_df["preds"].groupby("row_id").sum()
    else:
        train[f"preds{n}"] = oof_df["preds"]

# 各行の OOF の結果の数をカウントする
train["count_oof"] = len(c.params.pretrained) - train.isnull().sum(axis=1)  # + 4

# OOF の結果がない行を 0 埋め
train.fillna(0, inplace=True)

# OOF の結果をマージ
train["preds"] = 0
for col in preds_col:
    train["preds"] += train[col]

# 推論結果がないものは除外
train = train[train["preds"] != 0.0]

# 複数 OOF を加算している場合は、OOFの数で割る
train["preds"] = train["preds"] / train["count_oof"]

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
train

,time_id,target,preds0,preds1,preds2,preds3,preds4,count_oof,preds
row_id,,,,,,,,,
0_1,0,-0.300875,0.080811,-0.011368,0.076294,0.088074,0.065796,5,0.059921
0_2,0,-0.231040,0.110168,-0.024338,-0.068604,0.035095,-0.030533,5,0.004358
0_6,0,0.568807,0.026413,0.070984,0.001282,0.076111,0.038269,5,0.042612
0_7,0,-1.064780,0.051147,-0.043152,-0.012405,-0.068604,-0.084595,5,-0.031522
0_8,0,-0.531940,0.174683,0.110962,0.111816,0.063965,0.104126,5,0.113110
...,...,...,...,...,...,...,...,...,...
1219_3768,1219,0.033600,0.000000,0.000000,0.000000,0.000000,-0.017944,1,-0.017944
1219_3769,1219,-0.223264,0.000000,0.000000,0.000000,0.000000,0.011612,1,0.011612
1219_3770,1219,-0.559415,0.000000,0.000000,0.000000,0.000000,-0.083252,1,-0.083252


In [11]:
train["count_oof"].value_counts()

1    2196174
5     439243
Name: count_oof, dtype: int64

In [12]:
record_result(c, train, c.params.n_fold)

2022-02-26 20:46:30,577 [INFO] [get_score] Score: 0.11035


0.11034511207781533